# Initializations

In [1]:
import numpy as np
import pandas as pd

import sys, os, time
import glob

from matplotlib import pyplot as plt
%matplotlib inline

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

# widgets and interaction
from ipywidgets import FloatProgress
from IPython.display import display, clear_output

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from skimage.io import imread, imsave

import warnings
warnings.filterwarnings('ignore')

import gzip
import cPickle as pickle

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.utils as vutils
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import sys
sys.path.append("../ml-model/")

import models.dcgan as dcgan
import models.mlp as mlp
import models.dcgan_orig as do

# Set up data sources

In [5]:
dataroot = "/home/data/world-cities/"

In [6]:
import glob

files = glob.glob(dataroot + "/train/*/*.png")
files_df = []
for f in files:
    s = f.split("/")
    fname, cls = s[-1], s[-2]
    _,country,city,_,pop,lat,lon = fname.split("_")
    files_df.append((f, cls, country, pop))
    
files_df = pd.DataFrame(files_df, \
                columns=["filename", "class", "country", "population"])
files_df['city'] = files_df['filename'].apply(lambda s: os.path.basename(s).split("_")[2].replace("-"," "))
files_df['phase'] = 'train'

len(files_df)

24986

In [15]:
idx = np.random.choice(range(len(files_df)), size=int(0.8*len(files_df)), replace=False)
train_df = files_df.iloc[idx]
valid_df = files_df.iloc[list(set(range(len(files_df)))-set(idx))]

print len(train_df), len(valid_df)

19988 4998


In [85]:
train_df.to_csv(dataroot + "train.csv")
valid_df.to_csv(dataroot + "valid.csv")

#### Set up batching

In [21]:
pxCrop    = 448
classCol  = "class"
imageSize = 128
batchSize = 64
workers   = 4

In [22]:
import sys
sys.path.append("./../ml-model/pytorch_utils")
from loader_dataframe import ImageDataFrame, grayscale_loader, WeightedRandomSampler

from PIL import Image

def fn_rotate(img, max_angle=90):
    theta = np.random.randn()*max_angle
    return img.rotate(theta, expand=True)

train_dataset = ImageDataFrame(df=train_df, 
                         classCol=classCol,
                         loader=grayscale_loader,
                         transform=transforms.Compose([
                               transforms.RandomHorizontalFlip(),
                               transforms.CenterCrop(pxCrop),
                               transforms.Lambda(lambda img: fn_rotate(img)),
                               transforms.Scale(imageSize),
                               transforms.ToTensor()
                           ]))
test_dataset = ImageDataFrame(df=test_df, 
                         classCol=classCol,
                         loader=grayscale_loader,
                         transform=transforms.Compose([
                               transforms.CenterCrop(pxCrop),
                               transforms.Scale(imageSize),
                               transforms.ToTensor()
                           ]))
n_classes = len(train_dataset.classes)

weights = None #train_df['built pct']
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize,
                                         shuffle=True, 
                                         #sampler=WeightedRandomSampler(weights, len(train_df)),
                                         num_workers=int(workers))

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batchSize,
                                         shuffle=False, 
                                         num_workers=int(workers))

# Define model
- should work on arbitrary size images (multiple of 16)
- dual output: 0/1 for fake/real, vector of stats

In [23]:
nz = 100
nc = 1
ndf= 64
ngpu = 2
lr = 0.00005
n_extra_layers = 0
beta1 = 0.5
lam = 0.5

In [24]:
import torch
import torch.nn as nn
import torch.nn.parallel

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class DCGAN_D_DUAL(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu, n_extra_layers=0, n_classes=1):
        super(DCGAN_D_DUAL, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        features = nn.Sequential()
        # input is nc x isize x isize
        features.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        features.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            features.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            features.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            features.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            features.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            features.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            features.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        features.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, cndf, 4, 1, 0, bias=False))
        self.features = features
        self.flat_fts = self.get_flat_fts((nc,isize,isize), self.features)

        self.classifier_src = nn.Sequential(
            nn.Linear(self.flat_fts, 100),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
        self.classifier_cls = nn.Sequential(
            nn.Linear(self.flat_fts, 100),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(100,n_classes),
            nn.Softmax()
        )
        
    def get_flat_fts(self, in_size, fts):
        f = fts(Variable(torch.ones(1,*in_size)))
        return int(np.prod(f.size()[1:]))
    
    def forward(self, x):
        fts = self.features(x)
        flat_fts = fts.view(-1, self.flat_fts)
        out1 = self.classifier1(flat_fts)
        out2 = self.classifier2(flat_fts)
        return out1, out2
    
    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            fts = nn.parallel.data_parallel(self.features, input, range(self.ngpu))
            flat_fts = fts.view(-1, self.flat_fts)
            out_src= nn.parallel.data_parallel(self.classifier_src, flat_fts, range(self.ngpu))
            out_cls= nn.parallel.data_parallel(self.classifier_cls, flat_fts, range(self.ngpu))            
        else: 
            fts = self.features(input)
            out_src= self.classifier_src(fts)
            out_cls= self.classifier_cls(fts)
                
        return out_src, out_cls


In [25]:
net = DCGAN_D_DUAL(imageSize, nz, nc, ndf, ngpu, n_extra_layers, n_classes=n_classes)
net.apply(weights_init)


DCGAN_D_DUAL (
  (features): Sequential (
    (initial.conv.1-64): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial.relu.64): LeakyReLU (0.2, inplace)
    (pyramid.64-128.conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): LeakyReLU (0.2, inplace)
    (pyramid.128-256.conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.256.relu): LeakyReLU (0.2, inplace)
    (pyramid.256-512.conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.512.batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.512.relu): LeakyReLU (0.2, inplace)
    (pyramid.512-1024.conv): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), paddi

# Train model

In [77]:
input = torch.FloatTensor(batchSize, nc, imageSize, imageSize)
label = torch.LongTensor(batchSize, nc, imageSize, imageSize)
source_label = torch.FloatTensor(batchSize)
one = torch.FloatTensor([1])
mone = one * -1

if torch.cuda.is_available():
    net.cuda()
    input = input.cuda()
    label, source_label = label.cuda(), source_label.cuda()
    one, mone = one.cuda(), mone.cuda()

input = Variable(input)
label = Variable(label, requires_grad=False)
source_label = Variable(source_label, requires_grad=False)

    
# setup optimizer
optimizer = optim.Adam(net.parameters(), lr=lr, betas=(beta1, 0.999))

# optimization criteria
criterion_bce = nn.BCELoss()
criterion_mse = nn.MSELoss()
criterion_cls = nn.CrossEntropyLoss()


In [79]:
for i, (imgs, labs) in enumerate(train_dataloader):  
    batch_size = imgs.size(0)
    input.data.resize_(imgs.size()).copy_(imgs)
    label.data.resize_(labs.size()).copy_(labs)
    source_label.data.resize_(batch_size).fill_(0)
    break

In [81]:
out_src, out_sts = net(input)

loss_src = criterion_bce(out_src, source_label)
loss_sts = criterion_cls(out_sts, label)
loss = loss_src + lam * loss_sts

In [ ]:
for epoch in range(num_epochs):
    
    # Train for this epoch
    
    net.train(True)
    for i, (imgs, labs) in enumerate(train_dataloader):        
        batch_size = imgs.size(0)
        input.data.resize_(imgs.size()).copy_(imgs)
        label.data.resize_(labs.size()).copy_(labs)
        source_label.data.resize_(batch_size).fill_(0)

        # Forward + Backward + Optimize
        net.zero_grad()
        out_src, out_sts = net(input)
        loss_src = criterion_bce(out_src, source_label)
        loss_sts = criterion_cls(out_sts, label)
        loss = loss_src + lam * loss_sts        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    
    # Test for this epoch
    net.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0
    total = 0
    for imgs, labs in test_dataloader:
        imgs = Variable(imgs)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

    # checkpoint best model
    torch.save(netD.state_dict(), '{0}/netD_epoch_{1}.pth'.format(opt.experiment, epoch))


In [ ]:
net.train(mode=True)